In [ ]:
"""
Copyright (C) 2022 King Saud University, Saudi Arabia
SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License"); you may not use
this file except in compliance with the License. You may obtain a copy of the
License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed
under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR
CONDITIONS OF ANY KIND, either express or implied. See the License for the
specific language governing permissions and limitations under the License.

Author:  Hamdi Altaheri
"""

import math
import numpy as np
import pandas as pd
import pywt
import tensorflow as tf
from tensorflow.keras import layers, models, Input, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense
from tensorflow.keras.layers import multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda, Dropout, MultiHeadAttention, LayerNormalization

#########################
# Advanced Attention Blocks
#########################
def attention_block(in_layer, attention_model, ratio=8, residual=False, apply_to_input=True):
    in_sh = in_layer.shape  # shape of the input tensor
    in_len = len(in_sh)
    expanded_axis = 2  # default expansion axis for 3D -> 4D conversion

    if attention_model == 'mha':   # Multi-head self attention layer
        if in_len > 3:
            in_layer = Reshape((in_sh[1], -1))(in_layer)
        out_layer = mha_block(in_layer)
    elif attention_model == 'mhla':  # Multi-head local self-attention layer
        if in_len > 3:
            in_layer = Reshape((in_sh[1], -1))(in_layer)
        out_layer = mha_block(in_layer, vanilla=False)
    elif attention_model == 'se':   # Squeeze-and-excitation layer
        if in_len < 4:
            in_layer = tf.expand_dims(in_layer, axis=expanded_axis)
        out_layer = se_block(in_layer, ratio, residual, apply_to_input)
    elif attention_model == 'cbam': # Convolutional Block Attention Module
        if in_len < 4:
            in_layer = tf.expand_dims(in_layer, axis=expanded_axis)
        out_layer = cbam_block(in_layer, ratio=ratio, residual=residual)
    else:
        raise Exception("'{}' is not supported attention module!".format(attention_model))

    if in_len == 3 and len(out_layer.shape) == 4:
        out_layer = tf.squeeze(out_layer, expanded_axis)
    elif in_len == 4 and len(out_layer.shape) == 3:
        out_layer = Reshape((in_sh[1], in_sh[2], in_sh[3]))(out_layer)
    return out_layer

def mha_block(input_feature, key_dim=8, num_heads=2, dropout=0.5, vanilla=True):
    """Multi-head self Attention block."""
    x = LayerNormalization(epsilon=1e-6)(input_feature)
    if vanilla:
        x = MultiHeadAttention(key_dim=key_dim, num_heads=num_heads, dropout=dropout)(x, x)
    else:
        NUM_PATCHES = input_feature.shape[1]
        diag_attn_mask = 1 - tf.eye(NUM_PATCHES)
        diag_attn_mask = tf.cast([diag_attn_mask], dtype=tf.int8)
        x = MultiHeadAttention_LSA(key_dim=key_dim, num_heads=num_heads, dropout=dropout)(x, x, attention_mask=diag_attn_mask)
    x = Dropout(0.3)(x)
    mha_feature = Add()([input_feature, x])
    return mha_feature

class MultiHeadAttention_LSA(tf.keras.layers.MultiHeadAttention):
    """Local Multi-head Self Attention block (LSA)."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.tau = tf.Variable(math.sqrt(float(self._key_dim)), trainable=True)

    def _compute_attention(self, query, key, value, attention_mask=None, training=None):
        query = tf.multiply(query, 1.0 / self.tau)
        attention_scores = tf.einsum(self._dot_product_equation, key, query)
        attention_scores = self._masked_softmax(attention_scores, attention_mask)
        attention_scores_dropout = self._dropout_layer(attention_scores, training=training)
        attention_output = tf.einsum(self._combine_equation, attention_scores_dropout, value)
        return attention_output, attention_scores

def se_block(input_feature, ratio=8, residual=False, apply_to_input=True):
    """Squeeze-and-Excitation (SE) block."""
    channel_axis = 1 if tf.keras.backend.image_data_format() == "channels_first" else -1
    channel = input_feature.shape[channel_axis]
    se_feature = GlobalAveragePooling2D()(input_feature)
    se_feature = Reshape((1, 1, channel))(se_feature)
    if ratio != 0:
        se_feature = Dense(channel // ratio, activation='relu',
                           kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(se_feature)
    se_feature = Dense(channel, activation='sigmoid',
                       kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(se_feature)
    if tf.keras.backend.image_data_format() == 'channels_first':
        se_feature = Permute((3, 1, 2))(se_feature)
    if apply_to_input:
        se_feature = multiply([input_feature, se_feature])
    if residual:
        se_feature = Add()([se_feature, input_feature])
    return se_feature

def cbam_block(input_feature, ratio=8, residual=False):
    """Convolutional Block Attention Module (CBAM) block."""
    cbam_feature = channel_attention(input_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    if residual:
        cbam_feature = Add()([input_feature, cbam_feature])
    return cbam_feature

def channel_attention(input_feature, ratio=8):
    channel_axis = 1 if tf.keras.backend.image_data_format() == "channels_first" else -1
    channel = input_feature.shape[channel_axis]
    shared_layer_one = Dense(channel // ratio, activation='relu',
                             kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
    shared_layer_two = Dense(channel, kernel_initializer='he_normal',
                             use_bias=True, bias_initializer='zeros')

    avg_pool = GlobalAveragePooling2D()(input_feature)
    avg_pool = Reshape((1, 1, channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1, 1, channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    cbam_feature = Add()([avg_pool, max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)

    if tf.keras.backend.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)

    return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
    kernel_size = 7
    if tf.keras.backend.image_data_format() == "channels_first":
        cbam_feature = Permute((2, 3, 1))(input_feature)
    else:
        cbam_feature = input_feature
    avg_pool = Lambda(lambda x: tf.reduce_mean(x, axis=3, keepdims=True))(cbam_feature)
    max_pool = Lambda(lambda x: tf.reduce_max(x, axis=3, keepdims=True))(cbam_feature)
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    cbam_feature = Conv2D(filters=1, kernel_size=kernel_size, strides=1, padding='same',
                          activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(concat)
    if tf.keras.backend.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
    return multiply([input_feature, cbam_feature])


#########################
# Original Data Loading & Preprocessing
#########################
data_path = '/kaggle/input/dataset123321/SEED-VIG_processed (1).csv'
df = pd.read_csv(data_path)

print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())
print("\nDataset Info:")
print(df.info())
print("\nSummary Statistics:")
print(df.describe())
print("\nMissing values per column:")
print(df.isnull().sum())
print("\nUnique subjects (subindex):")
print(df['subindex'].unique())
print("\nClass distribution (substate):")
print(df['substate'].value_counts())

# Prepare raw EEG data: features and labels
X = df.drop(columns=['subindex', 'substate']).values
y = df['substate'].values
X = X.reshape(X.shape[0], 17, 384)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#########################
# Continuous Wavelet Transform Preprocessing
#########################
def apply_cwt(X, scales, wavelet):
    n_samples, n_channels, n_timepoints = X.shape
    transformed = []
    for i in range(n_samples):
        sample = X[i]
        channel_cwt = []
        for ch in range(n_channels):
            coeffs, _ = pywt.cwt(sample[ch], scales, wavelet)
            channel_cwt.append(coeffs)
        sample_cwt = np.stack(channel_cwt, axis=-1)
        transformed.append(sample_cwt)
    return np.array(transformed)

scales = np.arange(1, 31)  # 30 scales
wavelet = 'morl'
X_train_cwt = apply_cwt(X_train, scales, wavelet)
X_test_cwt  = apply_cwt(X_test, scales, wavelet)
print("Transformed training data shape:", X_train_cwt.shape)  # Expected: (n_samples, 30, 384, 17)
print("Transformed test data shape:", X_test_cwt.shape)

#########################
# Adaptive Feature Fusion Module (previous implementation)
#########################
def adaptive_feature_fusion_module_2d(inputs, filters):
    branch1 = layers.Conv2D(filters, kernel_size=(3, 3), padding='same', activation='relu',
                              kernel_regularizer=regularizers.l2(0.001))(inputs)
    branch1 = layers.BatchNormalization()(branch1)
    branch1 = layers.SpatialDropout2D(0.3)(branch1)

    branch2 = layers.Conv2D(filters, kernel_size=(5, 5), padding='same', activation='relu',
                              kernel_regularizer=regularizers.l2(0.001))(inputs)
    branch2 = layers.BatchNormalization()(branch2)
    branch2 = layers.SpatialDropout2D(0.3)(branch2)

    concat = layers.Concatenate()([branch1, branch2])
    fused = layers.Conv2D(filters, kernel_size=(1, 1), padding='same', activation='relu',
                          kernel_regularizer=regularizers.l2(0.001))(concat)
    fused = layers.BatchNormalization()(fused)
    out = layers.Add()([inputs, fused])
    return out

#########################
# Build the Model with Advanced Attention Blocks
#########################
def build_model_advanced(input_shape, attention_type='cbam'):
    """
    Build a CNN model that uses advanced attention modules.

    Parameters:
      - input_shape: shape of the input data (e.g., (30, 384, 17))
      - attention_type: one of 'cbam', 'se', 'mha', 'mhla'
    """
    inputs = Input(shape=input_shape)
    x = layers.GaussianNoise(0.05)(inputs)

    # Initial convolutional block
    x = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.SpatialDropout2D(0.3)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Apply advanced attention block
    x = attention_block(x, attention_model=attention_type, ratio=8, residual=True, apply_to_input=True)

    # Adaptive Feature Fusion
    x_aff = adaptive_feature_fusion_module_2d(x, 64)
    x = layers.Add()([x, x_aff])

    # Additional convolutional block
    x = layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.SpatialDropout2D(0.3)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Optionally, add another advanced attention block here
    x = attention_block(x, attention_model=attention_type, ratio=8, residual=True, apply_to_input=True)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.7)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build and summarize the model with advanced attention (using 'cbam' by default)
model_advanced = build_model_advanced((30, 384, 17), attention_type='cbam')
model_advanced.summary()

#########################
# Train the Model
#########################
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
history = model_advanced.fit(X_train_cwt, y_train, epochs=100, batch_size=32,
                             validation_data=(X_test_cwt, y_test),
                             callbacks=[reduce_lr])

#########################
# Evaluate the Model
#########################
y_pred = (model_advanced.predict(X_test_cwt) > 0.5).astype("int32")
test_accuracy = accuracy_score(y_test, y_pred)

print("\nTrain Accuracy:", history.history['accuracy'][-1])
print("\nValidation Accuracy:", history.history['val_accuracy'][-1])
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nTest Accuracy:", test_accuracy)


Dataset Shape: (4566, 6530)

First 5 rows:
   EEG_channel_1_t1  EEG_channel_1_t2  EEG_channel_1_t3  EEG_channel_1_t4  \
0         -3.457432        -15.610134        -28.209033        -16.500817   
1         12.856846          7.200323          4.795857         13.225275   
2         13.143232          3.307855          8.480391          7.013710   
3         30.523582         28.615215         41.685072         28.333728   
4         -8.910801        -10.594044        -30.384980        -10.884113   

   EEG_channel_1_t5  EEG_channel_1_t6  EEG_channel_1_t7  EEG_channel_1_t8  \
0        -16.546740        -24.126600        -24.409480         -5.469685   
1          3.833067         15.926209         11.693700          9.498457   
2          2.753549         -6.774113          7.940243          2.431553   
3         25.406661         16.704956         18.192949         29.348514   
4         -2.765033         -6.931607         -2.220146         -5.399940   

   EEG_channel_1_t9  EEG_channe

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 30, 384, 17)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gaussian_noise            │ (None, 30, 384, 17)    │              0 │ input_layer[0][0]      │
│ (GaussianNoise)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 30, 384, 64)    │          9,856 │ gaussian_noise[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 30, 384, 64)    │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ spatial_dropout2d         │ (None, 30, 384, 64)    │              0 │ batch_normalization[0… │
│ (SpatialDropout2D)        │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 15, 192, 64)    │              0 │ spatial_dropout2d[0][… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 64)             │              0 │ max_pooling2d[0][0]    │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling2d      │ (None, 64)             │              0 │ max_pooling2d[0][0]    │
│ (GlobalMaxPooling2D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 1, 1, 64)       │              0 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_1 (Reshape)       │ (None, 1, 1, 64)       │              0 │ global_max_pooling2d[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1, 1, 8)        │            520 │ reshape[0][0],         │
│                           │                        │                │ reshape_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1, 1, 64)       │            576 │ dense[0][0],           │
│                           │                        │                │ dense[1][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 1, 1, 64)       │              0 │ dense_1[0][0],         │
│                           │                        │                │ dense_1[1][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 1, 1, 64)       │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply (Multiply)       │ (None, 15, 192, 64)    │              0 │ max_pooling2d[0][0],   │
│                      

 Total params: 255,069 (996.36 KB)

 Trainable params: 254,301 (993.36 KB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 56s 277ms/step - accuracy: 0.6873 - loss: 1.0710 - val_accuracy: 0.8315 - val_loss: 0.8239 - learning_rate: 5.0000e-04
Epoch 2/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.8433 - loss: 0.7920 - val_accuracy: 0.7801 - val_loss: 0.8639 - learning_rate: 5.0000e-04
Epoch 3/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.8737 - loss: 0.6962 - val_accuracy: 0.7243 - val_loss: 0.9394 - learning_rate: 5.0000e-04
Epoch 4/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.9047 - loss: 0.6010 - val_accuracy: 0.8731 - val_loss: 0.6232 - learning_rate: 5.0000e-04
Epoch 5/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.9016 - loss: 0.5837 - val_accuracy: 0.8643 - val_loss: 0.6431 - learning_rate: 5.0000e-04
Epoch 6/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.9014 - loss: 0.5311 - val_accuracy: 0.7702 - val_loss: 0.8310 - learning_rate: 5.0000e-04
Epoch 7/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/s